In [1]:
import pandas as pd
import numpy as np
import torch
from bs4 import BeautifulSoup
import unicodedata
import string
import re
import nltk
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
from torch import nn

In [2]:
stopwords = nltk.corpus.stopwords.words('english')

In [3]:
all_letters = string.ascii_letters + " .,;'-"
df = pd.read_csv("./data/IMDB Dataset.csv")
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [4]:
def preprocess(text):
    cleanr = re.compile('<.*?>')
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = re.sub(cleanr, ' ', text)
    text = text.lower()
    text = regex.sub(' ', text)
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
    return text
def word_process(text, stopwords = False):
    text = text.split(" ")
    if stopwords : text = [word for word in text if word not in stopwords]
    return text

def encoding(text):
    text = text.split(" ")
    text = [word2index[word] for word in text]
    return text
    

In [7]:
text = df.copy()
text['review'] = df['review'].apply(preprocess)
text.head(2)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming t...,positive


In [10]:
all_words = word_process(" ".join(list(text['review'])))
words = list(set(all_words))
print(len(words))

100030


In [11]:
count_words = Counter(all_words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [12]:
vocab_to_int = {w:i for i, (w,c) in enumerate(sorted_words)}

In [13]:
reviews = list(text['review'])

In [14]:
reviews_encoded = []
for review in reviews:
    r = [vocab_to_int[w] for w in review.split()]
    reviews_encoded.append(r)
print (reviews_encoded[0:3])

[[28, 4, 1, 77, 2038, 46, 1050, 11, 100, 149, 41, 3061, 394, 20, 230, 29, 3174, 32, 25, 203, 14, 10, 6, 613, 47, 592, 17, 68, 1, 87, 148, 11, 3217, 68, 44, 3061, 13, 90, 5322, 2, 14820, 135, 4, 559, 61, 265, 8, 203, 37, 1, 647, 141, 1722, 68, 10, 6, 23, 3, 116, 16, 1, 7810, 2312, 40, 11302, 10, 116, 2571, 56, 5848, 17, 5442, 5, 1452, 371, 40, 559, 90, 6, 3784, 8, 1, 355, 356, 4, 1, 647, 7, 6, 433, 3061, 14, 11, 6, 1, 11473, 357, 5, 1, 14535, 6752, 2517, 1031, 50211, 7, 2684, 1399, 22, 22659, 518, 34, 4620, 2439, 4, 1, 1180, 115, 30, 1, 6931, 27, 2881, 11786, 2, 385, 50212, 36, 16327, 6, 23, 297, 22, 1, 4836, 2907, 518, 6, 340, 5, 107, 24380, 8063, 39249, 14536, 4993, 7691, 2426, 2, 52, 36, 43676, 324, 8981, 7236, 12281, 2, 8579, 31241, 25, 112, 223, 240, 9, 60, 132, 1, 280, 1315, 4, 1, 116, 6, 680, 5, 1, 192, 11, 7, 266, 115, 77, 274, 572, 21, 2982, 816, 182, 1287, 4124, 16, 2475, 1213, 816, 1418, 816, 863, 3061, 152, 21, 938, 184, 1, 87, 394, 9, 123, 209, 3217, 68, 14, 36, 1603, 7, 13

In [15]:
review_lens = [len(enc) for enc in reviews_encoded]

In [16]:
max(review_lens)

2492

In [17]:
np.mean(np.array(review_lens))


234.08298

In [18]:
labels = list(text['sentiment'])
encoded_labels = [1 if label =='positive' else 0 for label in labels]
encoded_labels = np.array(encoded_labels)

In [19]:
reviews_encoded = [ reviews_encoded[i] for i, l in enumerate(review_lens) if l<500 ]
encoded_labels = np.array([ encoded_labels[i] for i, l in enumerate(review_lens) if l< 500 ])

In [20]:
len(reviews_encoded), len(encoded_labels)

(45957, 45957)

In [21]:
def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features

In [22]:
features = pad_features(reviews_encoded, 500)
features.shape

In [23]:
split_frac = 0.8
len_feat = features.shape[0]
train_x = features[0:int(split_frac*len_feat)]
train_y = encoded_labels[0:int(split_frac*len_feat)]
remaining_x = features[int(split_frac*len_feat):]
remaining_y = encoded_labels[int(split_frac*len_feat):]
valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

In [24]:
#pytorch Datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50
# make sure to SHUFFLE your data

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [25]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length # batch_size sentences at once
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size # batch_size labels at once
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 500])
Sample input: 
 tensor([[   0,    0,    0,  ..., 1322,    1, 2164],
        [   0,    0,    0,  ...,  526,   44,    7],
        [   0,    0,    0,  ...,  726,    4, 4649],
        ...,
        [   0,    0,    0,  ..., 5531,    5,  845],
        [   0,    0,    0,  ..., 1306,    8, 3845],
        [   0,    0,    0,  ...,    3,   49,   28]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1], dtype=torch.int32)


In [26]:
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [27]:
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentLSTM(
  (embedding): Embedding(100031, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [40]:
# loss and optimization functions
lr=0.001
train_on_gpu = True

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
count = 0 
v_count = 0
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1
        inputs = inputs.type(torch.LongTensor)
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        count = count + 1
        if inputs.shape[0] != batch_size : continue


        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                inputs = inputs.type(torch.LongTensor)
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                if inputs.shape[0] != batch_size : continue
                v_count = v_count + 1
                
                
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.139125... Val Loss: 0.318243
Epoch: 1/4... Step: 200... Loss: 0.282121... Val Loss: 0.282419
Epoch: 1/4... Step: 300... Loss: 0.440695... Val Loss: 0.319842
Epoch: 1/4... Step: 400... Loss: 0.413789... Val Loss: 0.365040
Epoch: 1/4... Step: 500... Loss: 0.322951... Val Loss: 0.362384
Epoch: 1/4... Step: 600... Loss: 0.276541... Val Loss: 0.307061
Epoch: 1/4... Step: 700... Loss: 0.150527... Val Loss: 0.281822
Epoch: 2/4... Step: 800... Loss: 0.185218... Val Loss: 0.279230
Epoch: 2/4... Step: 900... Loss: 0.036196... Val Loss: 0.270710
Epoch: 2/4... Step: 1000... Loss: 0.117091... Val Loss: 0.272639
Epoch: 2/4... Step: 1100... Loss: 0.227487... Val Loss: 0.304202
Epoch: 2/4... Step: 1200... Loss: 0.155751... Val Loss: 0.268672
Epoch: 2/4... Step: 1300... Loss: 0.156565... Val Loss: 0.263972
Epoch: 2/4... Step: 1400... Loss: 0.098875... Val Loss: 0.268842
Epoch: 3/4... Step: 1500... Loss: 0.096785... Val Loss: 0.284112
Epoch: 3/4... Step: 1600... Loss: 

In [42]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])
    inputs = inputs.type(torch.LongTensor)

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    if inputs.shape[0] != batch_size : continue
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.440
Test accuracy: 0.865


In [46]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review("It's a bad movie")
print(test_ints)


# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)


# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())


def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    feature_tensor = feature_tensor.type(torch.LongTensor)
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

[[90, 3, 74, 15]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0 90  3 74 15]]
torch.Size([1, 200])


In [60]:
review_input = "There are a tons of characters, too many to follow. I keep forgetting who is who and is related to who. And then the multiple timelines make everything even more confusing. Not that it matters because there are basically no interesting storylines to follow."
predict(net, review_input)

Prediction value, pre-rounding: 0.066403
Negative review detected.
